In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install boto3

In [3]:
import support as support

In [2]:
import HR_analytics_project.support as nate

In [4]:
df_train_s3 = support.read_from_s3("hr_train_data","hranalyticsdata/new")

NameError: name 'AWS_ACCESS_KEY_ID' is not defined

In [10]:
df_train = df_train_s3.copy()
df_train.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0


In [11]:
print ('Train data:',df_train.shape)

Train data: (19158, 14)


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [14]:
 df_train['experience'].unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

In [15]:
df_train['company_size'].unique()

array([nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10/49',
       '100-500', '500-999'], dtype=object)

In [16]:
df_train.groupby([ 'education_level', 'enrolled_university']).count()

enrollee_id  city  \
education_level enrolled_university                      
Graduate        Full time course            2209  2209   
                Part time course             784   784   
                no_enrollment               8434  8434   
High School     Full time course             864   864   
                Part time course             166   166   
                no_enrollment                940   940   
Masters         Full time course             557   557   
                Part time course             204   204   
                no_enrollment               3547  3547   
Phd             Full time course              25    25   
                Part time course              12    12   
                no_enrollment                371   371   
Primary School  Full time course              33    33   
                Part time course              11    11   
                no_enrollment                255   255   

                                     city_development_index  gender  \
education_level enrolled_university                                   
Graduate        Full time course                       2209    1501   
                Part time course                        784     593   
                no_enrollment                          8434    6835   
High School     Full time course                        864     644   
                Part time course                        166     123   
                no_enrollment                           940     701   
Masters         Full time course                        557     366   
                Part time course                        204     135   
                no_enrollment                          3547    2793   
Phd             Full time course                         25      15   
                Part time course                         12       9   
                no_enrollment                           371     303   
Primary School  Full time course                         33      19   
                Part time course                         11       6   
                no_enrollment                           255     182   

                                     relevent_experience  major_discipline  \
education_level enrolled_university                                          
Graduate        Full time course                    2209              2200   
                Part time course                     784               784   
                no_enrollment                       8434              8422   
High School     Full time course                     864                 0   
                Part time course                     166                 0   
                no_enrollment                        940                 0   
Masters         Full time course                     557               556   
                Part time course                     204               203   
                no_enrollment                       3547              3543   
Phd             Full time course                      25                25   
                Part time course                      12                12   
                no_enrollment                        371               371   
Primary School  Full time course                      33                 0   
                Part time course                      11                 0   
                no_enrollment                        255                 0   

                                     experience  company_size  company_type  \
education_level enrolled_university                                           
Graduate        Full time course           2195          1040          1073   
                Part time course            784           587           571   
                no_enrollment              8418          6705          6511   
High School     Full time course            864           252           281   
                Part time course         

In [17]:
df_train['target'].unique()

array([1., 0.])

- Transformation:
    - Gender, major_discipline column: Create another category called "others"
    - enrolled_university, education_level, last_new_jobs: fill it with the mode
    - figure how to handle company_size and company_type = company_type(Other), experience (Mode), company_size (Other)
- Analysis:
    - A lot of analysis: create a presentation powerpoint, pdf
        - correlations
        - bar plot
        - pie charts
- ML Transformation
    - Label Encoder
    - One Hot Encoding
    - Embedding
    
- Model 
        

In [18]:
df_train['gender'].unique()

array(['Male', nan, 'Female', 'Other'], dtype=object)

In [19]:
df_train['last_new_job'].unique()

array(['1', '>4', '4', '3', 'never', '2', nan], dtype=object)

In [42]:
df_train.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [34]:
class Transformation:
    def __init__(self, df):
        self.df = df
        
    def replace_enrolled_university(self):
        uni_miss = self.df.loc[self.df['enrolled_university'].isna()]
        uni_miss['enrolled_university'] = self.df.loc[self.df['enrolled_university'].notna()]['enrolled_university'].mode()[0]
        self.df = pd.concat([self.df.loc[self.df['enrolled_university'].notna()], uni_miss], ignore_index=True)
    
    def replace_gender(self):
        gender_miss = self.df.loc[self.df['gender'].isna()]
        gender_miss['gender'] = 'Other'
        self.df = pd.concat([self.df.loc[self.df['gender'].notna()], gender_miss], ignore_index=True) 
    
    def replace_major_discipline(self):
        maj_disc_miss = self.df.loc[self.df['major_discipline'].isna()]
        maj_disc_miss['major_discipline'] = 'Other'
        self.df = pd.concat([self.df.loc[self.df['major_discipline'].notna()], maj_disc_miss], ignore_index=True)       
        
    def replace_education_level(self):
        edu_miss = self.df.loc[self.df['education_level'].isna()]
        edu_miss['education_level'] = self.df.loc[self.df['education_level'].notna()]['education_level'].mode()[0]
        self.df = pd.concat([self.df.loc[self.df['education_level'].notna()],edu_miss], ignore_index=True)
    
    def replace_last_new_job(self):
        job_miss  = self.df.loc[self.df['last_new_job'].isna()]
        job_miss['last_new_job'] = self.df.loc[self.df['last_new_job'].notna()]['last_new_job'].mode()[0]
        self.df = pd.concat([self.df.loc[self.df['last_new_job'].notna()],job_miss], ignore_index=True)
    
    def replace_company_type(self):
        type_miss = self.df.loc[self.df['company_type'].isna()]
        type_miss['company_type'] = 'Other'
        self.df = pd.concat([self.df.loc[self.df['company_type'].notna()], type_miss], ignore_index=True)
    
    def replace_company_size(self):
        size_miss = self.df.loc[self.df['company_size'].isna()]
        size_miss['company_size'] = 'Other'
        self.df = pd.concat([self.df.loc[self.df['company_size'].notna()], size_miss], ignore_index=True)
    
    def replace_experience(self):
        exp_miss = self.df.loc[self.df['experience'].isna()]
        exp_miss['experience'] = self.df.loc[self.df['experience'].notna()]['experience'].mode()[0]
        self.df = pd.concat([self.df.loc[self.df['experience'].notna()], exp_miss], ignore_index=True) 
        
    
    def run_process(self):
        self.replace_enrolled_university()
        self.replace_gender()
        self.replace_major_discipline()
        self.replace_education_level()
        self.replace_last_new_job()
        self.replace_company_type()
        self.replace_company_size()
        self.replace_experience()
        
        return self.df
    

In [40]:
import support

In [2]:
tran = support.Transformation(df_train)

NameError: name 'support' is not defined

In [36]:
cf = tran.run_process()

In [37]:
cf.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [41]:
cf = cf.drop("enrollee_id", axis=1)

In [38]:
pip install pivottablejs


  Using cached pivottablejs-0.9.0-py2.py3-none-any.whl (4.7 kB)


In [39]:
from pivottablejs import pivot_ui

pivot_ui(cf)

<img src="image.jpg"/>

In [ ]:
#pip install git+https://github.com/shakedzy/dython.git

In [ ]:
#from dython.nominal import associations

In [43]:
support.labeled_barplot(cf,"company_type",perc=True)

AttributeError: module 'support' has no attribute 'labeled_barplot'

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df_train.corr(), annot=True, cmap="YlGnBu", linewidths=.6)

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot (x='education_level',y='training_hours', data=df_train, label='blue')

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot (x='education_level',y='city_development_index', data=df_train, label='blue')

In [4]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 280C-2769

 Directory of C:\Users\Nathaniel\Downloads

17/07/2023  16:30    <DIR>          .
15/07/2023  23:48    <DIR>          ..
13/07/2023  21:25    <DIR>          .ipynb_checkpoints
13/07/2023  13:29    <DIR>          __pycache__
15/06/2022  11:50            84,199 1437923016_411__winequality-red.csv
23/03/2023  16:25         2,241,160 20220954970010.pdf
04/04/2023  11:25               154 36471 (1).trymyui
30/03/2023  21:21               154 36471.trymyui
19/01/2023  09:39        12,225,905 5c7825d2-6814-40c7-aefe-3d0f3d6f22e7.csv
28/03/2022  15:43             3,788 abtest.csv
11/07/2023  19:58         1,375,337 Academic Transcript of OKAFOR Nathaniel Chukwudimegwu (E-Transcript).pdf
07/03/2023  04:14           625,786 Acceptance Form.docx
13/07/2023  21:25           996,875 Additional_CaseStudy_German_Credit.ipynb
27/05/2023  06:52         1,531,464 Additional_CaseStudy_Loan_Delinquent.ipynb
10/02/2022  13:31            1

In [3]:
cp HR-Analytics.ipynb "/http://localhost:8888/tree/Downloads/HR_analytics_project/"

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
pwd

'C:\\Users\\Nathaniel\\Downloads\\HR_analytics_project'

In [9]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 280C-2769

 Directory of C:\Users\Nathaniel\Downloads\HR_analytics_project

17/07/2023  17:24    <DIR>          .
17/07/2023  16:52    <DIR>          ..
17/07/2023  16:56    <DIR>          .ipynb_checkpoints
17/07/2023  17:03    <DIR>          __pycache__
17/07/2023  17:24            59,570 HR-Analytics.ipynb
17/07/2023  16:56             5,178 support.py
16/07/2023  13:07               781 Untitled.ipynb
               3 File(s)         65,529 bytes
               4 Dir(s)  382,411,390,976 bytes free


In [10]:
cp HR-Analytics.ipynb "/Users/Nathaniel/PycharmProjects\HR_Recruitment_Analysis\"

SyntaxError: invalid syntax (4094667560.py, line 1)